In [1]:
#This notebook indexes all ip addressed in a large set to group them by subnet in paralell


from ipwhois import IPWhois
from pprint import pprint
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import socket, struct
from collections import OrderedDict
import os
import json
import re
import time

In [2]:
def parseCIDR(alpha):
    ip=alpha[:alpha.find('/')]
    mask=alpha[alpha.find('/')+1:]
    return (ip, int(mask))
    
def dottedQuadToNum(ip):
    "convert decimal dotted quad string to long integer"
    return struct.unpack('!L',socket.inet_aton(ip))[0]

def numToDottedQuad(n):
    "convert long int to dotted quad string"
    return socket.inet_ntoa(struct.pack('!L',n))
      
def makeMask(n):
    "return a mask of n bits as a long integer"
    return (1 << 32-n)-1

def ipToNetAndMask(ip):
    "returns tuple (network, host) dotted-quad addresses given IP and mask size"
    if (len(ip) > 18):
        ip=ip[:ip.find(',')]
    network,mask = parseCIDR(ip)
    n = dottedQuadToNum(network)
    m = makeMask(mask)

    host = n & m
    net = n - host

    return numToDottedQuad(net), mask

def toNet(n,maskbits):
    "returns tuple (network, host) dotted-quad addresses given IP and mask size"

    m = makeMask(maskbits)

    host = n & m
    net = n - host

    return numToDottedQuad(net)

In [3]:
date_path = '/data/data/2019-06-07/'
enrichment=date_path+'enrichment/'
stream = 'traceroute'
index=dict.fromkeys(os.listdir(enrichment))
for x in index:
    index[x]=os.listdir(enrichment+x)

#For each json in the "stream" folder for a particular date...
listofips=[]
for w in index[stream]:
    temp_json=json.load(open(enrichment+stream+'/'+w))
    if type(temp_json["data"]) == list:
        for route in temp_json["data"]:
            split=route["traceroute"].splitlines()
            for x in split:
                regIP=re.findall("([(]\d{1,3}[.]\d{1,3}[.]\d{1,3}[.]\d{1,3}[)])",x)
                for s in regIP:
                    listofips.append(s[1:len(s)-1])
                    break

#listofips=listofips[:31750]
print(len(listofips))

31750


In [4]:
subnetDir={}
iterCount=0
for test in listofips:
    iterCount+=1
    if (iterCount % 317 == 0):
        print (iterCount/31700)
        print(len(subnetDir))
    testKey=dottedQuadToNum(test)
    found=False
    for existingSub, existingMask in subnetDir.items():
        if toNet(testKey, existingMask) == existingSub:
            found=True
            break

    if not found:
        try:
            newSubnet,newMask=ipToNetAndMask(IPWhois(test,allow_permutations=True).lookup_rdap()['network']['cidr'])
        except:
            print(test)
        subnetDir[newSubnet]=newMask

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipwhois/net.py:138: UserWarning: allow_permutations has been deprecated and will be removed. It is no longer needed, due to the deprecation of asn_alts, and the addition of the asn_methods argument.
  warn('allow_permutations has been deprecated and will be removed. '


0.01
56
0.02
85
113.217.247.100
58.229.15.176
39.115.132.85
1.255.25.118
113.217.238.162
113.217.255.33
113.217.255.33
113.217.247.100
113.217.247.6
0.03
128
0.04
169
202.93.95.142
158.205.192.210
158.205.132.154
158.205.102.188
0.05
197
0.06
221
202.232.9.102
0.07
249
133.152.43.29
210.138.107.26
210.138.107.22
133.152.13.70
133.152.13.66
0.08
274
178.22.78.1
0.09
301
0.1
326
103.24.8.4
121.189.2.189
112.174.80.177
112.174.83.1
112.174.83.161
203.234.255.14
211.115.200.238
61.250.127.106
150.107.71.218
150.107.71.218
150.107.71.214
103.24.8.3
103.24.8.3
0.11
345
183.111.134.78
121.189.2.189
121.189.2.189
112.174.83.97
112.174.23.42
112.174.63.66
112.188.241.158
0.12
362
183.111.27.116
121.189.2.189
112.174.80.173
112.174.27.230
112.174.67.94
121.156.125.221
183.111.14.6
183.111.14.2
0.13
388
125.209.214.79
0.14
402
0.15
420
127.0.0.1
127.0.0.1
158.205.192.110
0.16
441
0.17
455
0.18
470
0.19
495
0.2
517
0.21
535
0.22
558
87.239.191.182
210.115.155.100
121.189.2.189
121.189.2.189
112.17

In [6]:
jsonWrite = json.dumps(subnetDir)
f = open("subnetDirectory.json","w")
f.write(jsonWrite)
f.close()